# Genome module

This module is there to represent and store genomic annotations, sequences etc. If you have ever used R's `GenomicFeatures` and `TxDb` packages you will feel right at home. This is basically a python replication of some of the functionalities.

In this notebook I will also introduce the `ranges` submodule in greater detail as it is used by the `Genome` instance to query the database.

## Database basics

The `Genome` class instance stores all the annotations in a SQL database, this can be any database of your choosing. Ideally it will be the same one as your `KnowledgeBase` but you can definitely use this as a standalone feature if you want and store everything in a `SQLite` database (which is what we are going to do in this notebook).

The database connection is handled by `SQLAlchemy` in the backend. You do not need to be an expert at using SQL as a lot of these queries are handled by the package for you.

I will be using *Saccharomyces cerevisiae* genome for this example as it is small. I have manually downloaded the genome fasta and gtf file from ensembl beforehand and they are in the same folder as this notebook.

On my computer the creation of the database takes about a minute with the yeast genome. It will take longer with a larger genome like mouse or human but you have to this once per genome.

In [1]:
from ccm_benchmate.genome.genome import Genome
from ccm_benchmate.ranges.genomicranges import GenomicRange

from sqlalchemy import create_engine

In [2]:
genome_db=create_engine('sqlite:///genome.sqlite')

yeast=Genome(genome_fasta="Saccharomyces_cerevisiae.R64-1-1.dna.toplevel.fa",
             gtf="Saccharomyces_cerevisiae.R64-1-1.114.gtf",
             db_conn=genome_db, name="yeast", description="Yeast")

Found an existing genome with yeast, just setting things up, if this is an error re-initiate the class with a different name


Depending on the size of the genome and the annotation file database creation can take a while. The database will be created in the same folder as this notebook. You can also specify a different folder if you want to while you are generating the sqlite engine. This flexibility allows you to create the database wherever you want. It can be on the local machine or it can be a database server on the network. You can use any database that SQLAlchemy supports, such as MySQL, PostgreSQL, SQLite, etc.

The database schema looks like this:

![](../assets/genome_database.png)

You can query the database for genes, transcripts, exons, coding sequences, utrs and introns. You can also extract sequences based on specific ranges provided that you have specified a fasta file. Additionally, you can extract sequences from the transcriptome or the proteome, but you will need to provied speicicic genomic ranges for that.

In [3]:
# get all the genes
all_genes=yeast.genes()

In [4]:
type(all_genes)

ccm_benchmate.ranges.genomicranges.GenomicRangesDict

I think this is a good time to delve a little bit deeper into `GenomicRanges`, `GenomicRangesList` and `GenomicRangesDict`. As we can see we have `GenomicRangesDict` instance here where all the keys are gene ids and the values are `GenomicRanges` instances. This is only true for the genes query. If you have queried transcripts the genomicrangesdict will have the transcripts grouped by genes and all other will have them grouped by transcripts.

In [5]:
ids=list(all_genes.keys())[0:10]

In [6]:
ids

['YDL246C',
 'YDL243C',
 'YDR387C',
 'YDL094C',
 'YDR438W',
 'YDR523C',
 'YDR542W',
 'YDR492W',
 'YDR018C',
 'YDL189W']

In [7]:
all_genes[ids[0]]

GenomicRange(IV:8683-9756(-))

I can do the same with all transcripts, exons, cdss, utrs and introns. No need to delve more into that.
but there is another interesting thing that these instances have and that's the annotation feature.

In [8]:
mygene=all_genes[ids[0]]
mygene.annotation

{'gene_id': 'YDL246C',
 'gene_name': 'SOR2',
 'gene_source': 'sgd',
 'gene_biotype': 'protein_coding',
 'db_id': 1}

Initially these come from the gtf file, so genes and transcripts have them but not others. That does not mean that you cannot add them (more on that later).

There is a strict hiearchical structure to the database. Genome > Chrom > Gene > Transcript > Exon > (CDS, 3' UTR, 5' UTR).

So if you want to get the 3' utr you need to get the gene, then the transcript. You can search introns, exons, cdss and utrs by transcript id and you can search trancripts by gene id.

There is another thing in there that you will need. The database is desgined to store multiple genomes, each different fasta/gtf combination is stored as a different genome. So you will need the database unique id to query for things. Well, I lied, I'm working on the latter part.

In [9]:
transcript=yeast.transcripts(gene_ids=mygene.annotation["db_id"])

In [10]:
len(transcript)

1

In [11]:
transcript

GenomicRangesDict(dict_items([('YDL246C', GenomicRangesList([GenomicRange(IV:8683-9756(-))]))]))

In [12]:
transcript.keys()

dict_keys(['YDL246C'])

In [13]:
transcript['YDL246C'][0]

GenomicRange(IV:8683-9756(-))

But wait, there is more. If you look at the annotations feature of the genomic ranges you will see that we also have the database ids in there.

In [14]:
transcript['YDL246C'][0].annotation

{'gene_id': 'YDL246C',
 'transcript_id': 'YDL246C_mRNA',
 'gene_name': 'SOR2',
 'gene_source': 'sgd',
 'gene_biotype': 'protein_coding',
 'transcript_name': 'SOR2',
 'transcript_source': 'sgd',
 'transcript_biotype': 'protein_coding',
 'tag': 'Ensembl_canonical',
 'db_id': 1}

That means I can pass arbitrary ids to any of the queries and it will return that specific feature with that id. This is useful if you want to get a specific feature without having to query the whole database.

In [15]:
ids=[1,3,4,5,6,78]
exons=yeast.exons(ids=ids)

In [16]:
exons

GenomicRangesDict(dict_items([('YDL246C_mRNA', GenomicRangesList([GenomicRange(IV:8683-9756(-))])), ('YDR387C_mRNA', GenomicRangesList([GenomicRange(IV:1248154-1249821(-))])), ('YDL094C_mRNA', GenomicRangesList([GenomicRange(IV:289572-290081(-))])), ('YDR438W_mRNA', GenomicRangesList([GenomicRange(IV:1338274-1339386(+))])), ('YDR523C_mRNA', GenomicRangesList([GenomicRange(IV:1485566-1487038(-))])), ('YDR425W_mRNA', GenomicRangesList([GenomicRange(IV:1320064-1321941(+))]))]))

For transcripts you can pass gene ids, for all other features you can pass transcript ids and it will return the features that you asked for those transcripts.

In [17]:
exons=yeast.exons(transcript_ids=ids)

In [18]:
exons

GenomicRangesDict(dict_items([('YDL246C_mRNA', GenomicRangesList([GenomicRange(IV:8683-9756(-))])), ('YDR387C_mRNA', GenomicRangesList([GenomicRange(IV:1248154-1249821(-))])), ('YDL094C_mRNA', GenomicRangesList([GenomicRange(IV:289572-290081(-))])), ('YDR438W_mRNA', GenomicRangesList([GenomicRange(IV:1338274-1339386(+))])), ('YDR523C_mRNA', GenomicRangesList([GenomicRange(IV:1485566-1487038(-))])), ('snR13_snoRNA', GenomicRangesList([GenomicRange(IV:1402919-1403042(+))]))]))

last but not least you can get all the features that are within a specific genomic range. All you have to do is to create a `GenomicRange` instance and pass it to the query. This will return all the features that overlap with that range.

In [19]:
myrange=GenomicRange("I", 1000, 2000000, "+")

In [20]:
introns=yeast.introns(range=myrange)

In [21]:
introns

GenomicRangesDict(dict_items([('YAL003W_mRNA', GenomicRangesList([GenomicRange(I:142254-142619(+))])), ('tL(CAA)A_tRNA', GenomicRangesList([GenomicRange(I:181179-181210(+))])), ('tP(UGG)A_tRNA', GenomicRangesList([GenomicRange(I:139188-139218(+))])), ('YAL030W_mRNA', GenomicRangesList([GenomicRange(I:87388-87500(+))]))]))

And here is how you extract sequences:

In [23]:
myrange=GenomicRange("I", 100, 200, "+")
yeast.get_sequence(myrange)

'GCCAACCTGTCTCTCAACTTACCCTCCATTACCCTGCCTCCACTCGTTACCCTGTCCCATTCAACCATACCACTCCGAACCACCATCCATCCCTCTACTT'

In [24]:
#it's strand specific
myrange=GenomicRange("I", 100, 200, "-")
yeast.get_sequence(myrange)

'AAGTAGAGGGATGGATGGTGGTTCGGAGTGGTATGGTTGAATGGGACAGGGTAACGAGTGGAGGCAGGGTAATGGAGGGTAAGTTGAGAGACAGGTTGGC'

# A bit deeper into Ranges and GenomicRanges

under the ranges submodule we have a few classes that are used to represent genomic ranges and their annotations. The main class is `GenomicRange` which represents a single genomic range. It has the following attributes:

- `chrom`: the chromosome name
- `start`: the start position of the range
- `end`: the end position of the range
- `strand`: the strand of the range, either `+` or `-` or `*` for both strands

These are inspired by R's Ranges and GenomicRanges packages. The `GenomicRange` class has a few methods that allow you to manipulate the ranges, such as `shift`, `resize`, `intersect`, `union`, etc. You can also create a list of ranges using the `GenomicRangesList` class which is a simple list of `GenomicRange` instances.

Additionally, we have the `GenomicRangesDict` class which is a dictionary of `GenomicRange` or `GenomicRangesList` instances. This is used to store ranges grouped by some feature, such as genes or transcripts.

There are numerous methods that allow you to perform operations. For `GenomicRange` you can do things like: `shift`, `extend`, find whether it overlaps with another ranges, calculate distance between ranges using `distance`.

If you are using `GenomicRangesList` you can do things like you have list like methods like `append`, `extend`, `insert`, `remove`, `pop`. The main difference between a regular list is that you can only add another instance of a `GenomicRange` to it.

You can find overlapping ranges with another list, if not specified it will use itself and return the overllaping regions. You can `reduce` the r`GenomicRangesList` to a single `GenomicRange` instance by merging all the ranges and using the min start and max end positions.

`Coverage` method is very similar to read coverage, in that it will count the number of ranges that map to each position within the bounds of the `GenomicRangesList`. This is useful for visualizing the coverage of a specific region.

`GenomicRangesDict` is a dictionary of `GenomicRangesList` instances. It is used to store ranges grouped by some feature, such as genes or transcripts. You can access the ranges by their keys and perform operations on them just like you would with a regular dictionary. Again the only restriction is that the values must be a `GenomicRange` or `GenomicRangesList` instances.

There is more in the specific documentation of the `ranges` submodule.

**COMING SOON**: Currently there is no way to add to annotations of ranges and update the database or any way to search the annotations, like return all the protein coding transcripts etc. Those will be added very soon.